In [10]:
# Загрузим необходимые библиотеки и зафиксируем seed
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
from torch import nn
from torch import optim
from torchsummary import summary
from tqdm import trange
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import pandas as pd
import numpy as np
from  pytorch_lightning import seed_everything
from sklearn.base import BaseEstimator
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

seed_everything(42)

Seed set to 42


42

In [2]:
# набор функций для метрик и визуализации

def plot_train_process(train_loss, val_loss, train_accuracy, val_accuracy, title_suffix=''):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].set_title(' '.join(['Loss', title_suffix]))
    axes[0].plot(train_loss, label='train')
    axes[0].plot(val_loss, label='validation')
    axes[0].legend()

    axes[1].set_title(' '.join(['Validation accuracy', title_suffix]))
    axes[1].plot(train_accuracy, label='train')
    axes[1].plot(val_accuracy, label='validation')
    axes[1].legend()
    plt.show()

def visualize_and_save_results(model, model_name, X_train, X_test, y_train, y_test, out_dict):
    for data_name, X, y, model in [
    ('train', X_train, y_train, model),
    ('test', X_test, y_test, model)
    ]:
        if isinstance(model, BaseEstimator):
          try:
            proba = model.predict_proba(X)[:, 1]
          except AttributeError:
            proba = model.decision_function(X) # не у всех классификаторов есть predict_proba - для них можно считать decision function, значение метрики должно быть аналогичным.
            # Из документации: https://scikit-learn.org/stable/modules/model_evaluation.html#roc-auc-binary:~:text=Otherwise%2C%20we%20can%20use%20the%20non%2Dthresholded%20decision%20values

        elif isinstance(model, nn.Module):
            proba = model(X).detach().cpu().numpy()[:, 1]
        else:
            raise ValueError('Unrecognized model type')

        auc = roc_auc_score(y, proba)

        out_dict['{}_{}'.format(model_name, data_name)] = auc
        plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (data_name, auc))

    plt.plot([0, 1], [0, 1], '--', color='black',)
    plt.legend(fontsize='large')
    plt.title(model_name)
    plt.grid()
    return out_dict

def get_metrics(gs, out_dict: dict) -> dict:
    """Сбор параметров и метрик обученной модели

    Args:
        gs (_type_): Экземпляр класса GridSearchCV после обучения
        out_dict (dict): Словарь с метриками на тесте и лучшими параметрами модели

    Returns:
        dict: Обновленный словарь со средними метриками на обучающей и валидационной выборках
    """

    val_auc = gs.best_score_
    metrics_df = pd.DataFrame(gs.cv_results_)
    total_main_metrics = metrics_df[['mean_train_score', 'mean_test_score', 'std_test_score']]
    main_metrics_dict = total_main_metrics[total_main_metrics['mean_test_score'] == val_auc].to_dict()
    out_dict.update(**main_metrics_dict)
    return out_dict

___
### Исходный датафрейм

In [9]:
FILE_PATH = 'data/hackaton_result_dataset.xlsx'

df = pd.read_excel(FILE_PATH)
df[:3]

,model_annotation,human_markup,audio_path,label
0,давай по россии значит на коленях быстро блять...,давай проси значит на коленях быстро блять,s3://ap-training-set/xacaton_openstt/part_1/05...,1
1,ну разве можно так с телефоном поступает,ну что ну разве можно так с телефоном поступать,s3://ap-training-set/xacaton_openstt/part_1/05...,0
2,у меня нет с собой в полном адресе я щас дома ...,у меня нет с собой полного адреса я щас из дом...,s3://ap-training-set/xacaton_openstt/part_1/05...,0


___
### SBERT

In [10]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru",**{'padding':True, 'truncation':True, 'max_length':24, 'return_tensors':'pt'})
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

C:\Users\User\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\User\anaconda3\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [11]:
def mean_pooling(tokens, attention_mask):
    with torch.no_grad():
        output = model(**tokens)
    token_embeddings = output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

### Токенизация

In [12]:
df['tokens'] = df['model_annotation'].apply(lambda x: tokenizer(x, padding=True, truncation=True, max_length=24, return_tensors='pt'))

In [13]:
df['tokens'][0]

{'input_ids': tensor([[  101,  3823,   377,   378,   654, 44422,  3216,   660, 18568,  2547,
         51121,  1344,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### Эмбеддинги

In [14]:
tqdm.pandas()
df['emb'] = df['tokens'].progress_apply(lambda token: mean_pooling(token, token['attention_mask']))

100%|██████████████████████████████████████████████████████████████████████████████| 6508/6508 [43:16<00:00,  2.51it/s]


In [15]:
df['emb'][:3]

0    [[tensor(-0.3775), tensor(0.0096), tensor(-0.0...
1    [[tensor(-0.3522), tensor(-0.1911), tensor(0.9...
2    [[tensor(-0.2016), tensor(0.1401), tensor(-0.6...
Name: emb, dtype: object

___
### Сохранение датафрейма

In [6]:
df.to_pickle('data/df_emb.pkl')

OSError: Cannot save file into a non-existent directory: 'data'

___
### Загрузка датафрейма

In [5]:
df = pd.read_pickle('df_emb.pkl')

___
### Разделение выборки

In [7]:
X = df['emb']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [8]:
X_train_ser = X_train.apply(lambda x: torch.reshape(x, shape=(1024,)).numpy())
X_test_ser = X_test.apply(lambda x: torch.reshape(x, shape=(1024,)).numpy())
y_train_arr = y_train.to_numpy()
y_test_arr = y_test.to_numpy()

### Series -> array

In [11]:

def get_matrix(X: pd.Series) -> np.ndarray:
    X_train_matrix = np.zeros((X.shape[0], 1024))
    for i, array in enumerate(X):
        X_train_matrix[i] = array

    return X_train_matrix

X_train_matrix = get_matrix(X_train_ser)
X_test_matrix = get_matrix(X_test_ser)

___
### Классические модели

___
### `LinearSVC`

In [15]:
# Подберем параметры для LinearSVC
from sklearn.svm import LinearSVC

param_grid_LinearSVC_l2 = {
    "penalty" : ["l2"],
    'loss' : ["hinge","squared_hinge"],
    'C': [0.1,0.3,0.5,1,2],
    }

lsvc_clf = LinearSVC(max_iter = 10000)

gs_lsvc_l2 = GridSearchCV(estimator=lsvc_clf, param_grid=param_grid_LinearSVC_l2, cv=StratifiedKFold(5),return_train_score=True, scoring='roc_auc', n_jobs=2)
gs_lsvc_l2.fit(X_train_matrix, y_train_arr)

best_params_lsvc_l2 = gs_lsvc_l2.best_params_
best_score_lsvc_l2 = gs_lsvc_l2.best_score_

print ("Лучшие параметры", best_params_lsvc_l2)
print ("Лучшее значение метрики качества", best_score_lsvc_l2)

Лучшие параметры {'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}
Лучшее значение метрики качества {'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}


In [17]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(gs_lsvc_l2, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
0,0.879449,0.749444,0.008255


___
### `DecisionTreeClassifier`

In [16]:
# Подберем параметры для DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

param_grid_dtc = {
    'criterion': ["gini", "entropy", "log_loss"],
    'max_depth': [4,5,6,7,8,9,10],
    }

dtc = DecisionTreeClassifier()
grid_search_dtc = GridSearchCV(estimator = dtc,  param_grid = param_grid_dtc, cv = StratifiedKFold(5),
                                return_train_score=True, scoring = 'roc_auc', n_jobs=2)
grid_search_dtc.fit(X_train_matrix, y_train_arr)
best_params_dtc = grid_search_dtc.best_params_
best_score_dtc = grid_search_dtc.best_score_
print("Лучшие параметры:", best_params_dtc)
print ("Лучшее значение метрики качества:", best_score_dtc)

Лучшие параметры: {'criterion': 'gini', 'max_depth': 4}
Лучшее значение метрики качества: 0.6419694902327145


In [19]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(grid_search_dtc, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
0,0.715165,0.641969,0.017191


___
### `RandomForestClassifier`

In [20]:
# Подберем параметры для RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

param_grid_rfc1 = {
    #"min_samples_splitint": [10, 100, 1000],
    #'min_samples_leafint': [10, 100, 1000],
    'criterion': ["gini", "entropy", "log_loss"],
    'max_depth': [1,2,3],
    'n_estimators': [200, 300, 400]
    }

rfc = RandomForestClassifier()
grid_search_rfc1 = GridSearchCV(estimator = rfc,  param_grid = param_grid_rfc1, cv = StratifiedKFold(5),
                                return_train_score=True, scoring = 'roc_auc', n_jobs=2)
grid_search_rfc1.fit(X_train_matrix, y_train_arr)
best_params_rfc1 = grid_search_rfc1.best_params_
best_score_rfc1 = grid_search_rfc1.best_score_
print("Лучшие параметры:", best_params_rfc1)
print ("Лучшее значение метрики качества:", best_score_rfc1)

Лучшие параметры: {'criterion': 'log_loss', 'max_depth': 3, 'n_estimators': 300}
Лучшее значение метрики качества: 0.7342188086870225


In [22]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(grid_search_rfc1, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
25,0.807196,0.734219,0.010883


In [21]:
# При увеличении количества деревьев качество растет. Попробуем увеличить их количество
from sklearn.ensemble import RandomForestClassifier

param_grid_rfc2 = {
    'criterion': ["gini"],
    'max_depth': [3, 4, 5],
    'n_estimators': [400, 500, 600]
    }

rfc = RandomForestClassifier()
grid_search_rfc2 = GridSearchCV(estimator = rfc,  param_grid = param_grid_rfc2, cv = StratifiedKFold(5),
                                return_train_score=True, scoring = 'roc_auc', n_jobs=2)
grid_search_rfc2.fit(X_train_matrix, y_train_arr)
best_params_rfc2 = grid_search_rfc2.best_params_
best_score_rfc2 = grid_search_rfc2.best_score_
print("Лучшие параметры:", best_params_rfc2)
print ("Лучшее значение метрики качества:", best_score_rfc2)

Лучшие параметры: {'criterion': 'gini', 'max_depth': 5, 'n_estimators': 400}
Лучшее значение метрики качества: 0.7439753347983313


In [23]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(grid_search_rfc2, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить
# Модель начинает переобучатся. Дальнейшее увеличение деревьев не имеет смысла.

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
6,0.904582,0.743975,0.01133


___
### `GradientBoostingClassifier`

Лучшие параметры: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 1, 'n_estimators': 200}
Лучшее значение метрики качества: 0.7507587332826351

In [25]:
# Подберем параметры для GradientBoostingClassifier. Так как он достаточно ресурсоемкий
# будет подбирать параметры в несколько итераций
from sklearn.ensemble import GradientBoostingClassifier

param_grid_gbc = {
    "loss":["log_loss", "exponential"],
    "criterion": ["friedman_mse", "squared_error"],
    'learning_rate': [0.1],
    'max_depth': [1],
    'n_estimators': [200]
    }

gbc = GradientBoostingClassifier()
grid_search_gbc = GridSearchCV(estimator = gbc,  param_grid = param_grid_gbc, cv = StratifiedKFold(5),
                                return_train_score=True, scoring = 'roc_auc', n_jobs=2)
grid_search_gbc.fit(X_train_matrix, y_train_arr)
best_params_gbc = grid_search_gbc.best_params_
best_score_gbc = grid_search_gbc.best_score_
print("Лучшие параметры:", best_params_gbc)
print ("Лучшее значение метрики качества:", best_score_gbc)

Лучшие параметры: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 1, 'n_estimators': 200}
Лучшее значение метрики качества: 0.7507587332826351


In [26]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(grid_search_gbc, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
0,0.82014,0.750759,0.013945


In [27]:
# Попробуем увеличить глубину деревьев
from sklearn.ensemble import GradientBoostingClassifier

param_grid_gbc2 = {
    "loss":["log_loss"],
    "criterion": ["friedman_mse"],
    'learning_rate': [0.1],
    'max_depth': [2],
    'n_estimators': [200]
    }

gbc = GradientBoostingClassifier()
grid_search_gbc2 = GridSearchCV(estimator = gbc,  param_grid = param_grid_gbc2, cv = StratifiedKFold(5),
                                return_train_score=True, scoring = 'roc_auc', n_jobs=2)
grid_search_gbc2.fit(X_train_matrix, y_train_arr)
best_params_gbc2 = grid_search_gbc2.best_params_
best_score_gbc2 = grid_search_gbc2.best_score_
print("Лучшие параметры:", best_params_gbc2)
print ("Лучшее значение метрики качества:", best_score_gbc2)

Лучшие параметры: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 2, 'n_estimators': 200}
Лучшее значение метрики качества: 0.7607630331028634


In [30]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(grid_search_gbc2, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
0,0.920988,0.760763,0.012738


In [28]:
# Попробуем повысить learning_rate
from sklearn.ensemble import GradientBoostingClassifier

param_grid_gbc3 = {
    "loss":["log_loss"],
    "criterion": ["friedman_mse"],
    'learning_rate': [0.2],
    'max_depth': [1],
    'n_estimators': [200]
    }

gbc = GradientBoostingClassifier()
grid_search_gbc3 = GridSearchCV(estimator = gbc,  param_grid = param_grid_gbc3, cv = StratifiedKFold(5),
                                return_train_score=True, scoring = 'roc_auc', n_jobs=2)
grid_search_gbc3.fit(X_train_matrix, y_train_arr)
best_params_gbc3 = grid_search_gbc3.best_params_
best_score_gbc3 = grid_search_gbc3.best_score_
print("Лучшие параметры:", best_params_gbc3)
print ("Лучшее значение метрики качества:", best_score_gbc3)

Лучшие параметры: {'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'log_loss', 'max_depth': 1, 'n_estimators': 200}
Лучшее значение метрики качества: 0.7584698758307615


In [29]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(grid_search_gbc3, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить
# Качество ожидаемо снизилось

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
0,0.857784,0.75847,0.014361


In [31]:
# Попробуем увеличить глубину деревьев и их количество
from sklearn.ensemble import GradientBoostingClassifier

param_grid_gbc4 = {
    "loss":["log_loss"],
    "criterion": ["friedman_mse"],
    'learning_rate': [0.1],
    'max_depth': [3],
    'n_estimators': [300]
    }

gbc = GradientBoostingClassifier()
grid_search_gbc4 = GridSearchCV(estimator = gbc,  param_grid = param_grid_gbc4, cv = StratifiedKFold(5),
                                return_train_score=True, scoring = 'roc_auc', n_jobs=2)
grid_search_gbc4.fit(X_train_matrix, y_train_arr)
best_params_gbc4 = grid_search_gbc4.best_params_
best_score_gbc4 = grid_search_gbc4.best_score_
print("Лучшие параметры:", best_params_gbc4)
print ("Лучшее значение метрики качества:", best_score_gbc4)

Лучшие параметры: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 3, 'n_estimators': 300}
Лучшее значение метрики качества: 0.7622134256101687


In [32]:
# Проверим получившиеся метрики
test_val_dict = dict()
test_val_dict = get_metrics(grid_search_gbc4, test_val_dict)
print('Тестовые и валидационные метрики из GridSearchCV:')
pd.DataFrame(test_val_dict).drop_duplicates() #нам интересно в целом оценить метрики, поэтому дубликаты можно не выводить
# Модель переобучается. Дальнейшее повышение глубины и количества деревьев не имеет смысла

Тестовые и валидационные метрики из GridSearchCV:


,mean_train_score,mean_test_score,std_test_score
0,0.994923,0.762213,0.012588
